In [1]:
import torch
from torch.utils.data import DataLoader
from torch import nn, IntTensor, Tensor

In [2]:
import pandas as pd

In [3]:
from src.data.df_dataset import DfDataset

In [4]:
df = pd.read_csv("../data/output/with_factors.csv")

In [5]:
dataset = DfDataset(df, ["rna_loops"])

In [6]:
loop_sequence = df["rna_loops"]
loop_sequence = df["rna_loops"].apply(
    lambda x: 
)

SyntaxError: invalid syntax (3828123715.py, line 4)

In [7]:
loader = DataLoader(dataset, batch_size=128)

In [8]:
x, y = list(loader)[0]

In [9]:
loops = x["rna_loops"]

In [15]:
mapping = {
    "(": 0,
    ".": 1,
    ")": 2
}

encoded_loops = [
    list(map(mapping.get, loop))
    for loop in loops
]
encoded_loops = IntTensor(encoded_loops)

In [116]:
num_conv_layers = 2
embedding_size = 4
num_conv_filters = 3
filter_size = 3
input_size = 117
latent_size = 8

embedding = nn.Embedding(3, embedding_size)

encoder_layers = []

for i in range(num_conv_layers):
    if i == 0:
        layer_input_size = embedding_size
    else:
        layer_input_size = num_conv_filters

    encoder_layers += [
        nn.Conv1d(layer_input_size, num_conv_filters, filter_size),
        nn.ReLU(),
    ]

encoder_layers += [
    nn.Flatten(),
    nn.Linear(
        num_conv_filters * (input_size - num_conv_layers * (filter_size - 1)),
        latent_size,
    ),
    nn.ReLU(),
]

encoder = nn.Sequential(*encoder_layers)

mu = nn.Linear(latent_size, latent_size)
var = nn.Linear(latent_size, latent_size)

In [117]:
def reparameterize(mu: Tensor, logvar: Tensor) -> Tensor:
    """
    Reparameterization trick to sample from N(mu, var) from
    N(0,1).
    """
    std = torch.exp(0.5 * logvar)
    eps = torch.randn_like(std)
    return eps * std + mu

In [118]:
decoder_layers = []

decoder_layers += [
    nn.Linear(
        latent_size,
        num_conv_filters * (input_size - num_conv_layers * (filter_size - 1)),
    ),
    nn.ReLU(),
    nn.Unflatten(1, (num_conv_filters, input_size - num_conv_layers * (filter_size - 1))),
]

for _ in range(num_conv_layers-1):
    decoder_layers += [
        nn.ConvTranspose1d(num_conv_filters, num_conv_filters, filter_size),
        nn.ReLU(),
    ]

decoder_layers += [
    nn.ConvTranspose1d(num_conv_filters, embedding_size, filter_size),
]


decoder = nn.Sequential(*decoder_layers)

In [119]:
x = embedding(encoded_loops)  # (batch_size, seq_len, embedding_dim)
x = x.permute(0, 2, 1)  # (batch_size, embedding_dim, seq_len)

In [120]:
encoding = encoder(x)
mu = mu(encoding)
var = var(encoding)

In [121]:
decoder

Sequential(
  (0): Linear(in_features=8, out_features=339, bias=True)
  (1): ReLU()
  (2): Unflatten(dim=1, unflattened_size=(3, 113))
  (3): ConvTranspose1d(3, 3, kernel_size=(3,), stride=(1,))
  (4): ReLU()
  (5): ConvTranspose1d(3, 4, kernel_size=(3,), stride=(1,))
)

In [125]:
z = reparameterize(mu, var)
z = decoder(z)

In [129]:
z.shape, embedding(encoded_loops).shape

(torch.Size([128, 4, 117]), torch.Size([128, 117, 4]))

In [128]:
embedding.shape

AttributeError: 'Embedding' object has no attribute 'shape'

In [139]:
x.shape

torch.Size([128, 5, 115])

In [140]:
linear_layer = nn.Linear(
    5*(117 - 1 * (3 - 1)), 1
)

In [141]:
x = x.flatten(start_dim=1)

In [142]:
linear_layer(x).shape

RuntimeError: Placeholder storage has not been allocated on MPS device!

In [9]:
from src.modeling.cnn_vae import PredictionCNN
from torch import optim

In [10]:
model = PredictionCNN(
    optimizer=optim.Adam,
)

In [14]:
model._get_loss(list(loader)[0])

/Users/tobiaochsner/Documents/ETH/GeneticFitness/.venv/lib/python3.11/site-packages/lightning/pytorch/core/module.py:419: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(


tensor(0.9044, grad_fn=<AddBackward0>)

In [139]:
x, _ = list(loader)[0]

In [141]:
loops = x["rna_loops"]
encoded_loops = [list(map({"(": 0, ".": 1, ")": 2}.get, loop)) for loop in loops]
encoded_loops = IntTensor(encoded_loops)

In [142]:
encoded_loops

tensor([[1, 1, 1,  ..., 2, 2, 2],
        [1, 1, 1,  ..., 2, 2, 1],
        [1, 1, 1,  ..., 2, 2, 2],
        ...,
        [1, 1, 1,  ..., 2, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 2, 2, 2]], dtype=torch.int32)